# Patat : Plateforme Agnostique de Traitement et d'Analyse des Textes

### Notebook 1 : Création du Recueil Complet
En partant du Recueil Brut issu de la google spreadsheet observateurs

### Synopsis
- Chargement et Nettoyage du Recueil
- Importation et Parsing des articles
- Nettoyage et Sauvegarde du Recueil Complet

### To do
- Moins de NaN dans le recueil
- Meilleure prise en compte des NaN dans la suite

# 0 - Préliminaires

In [ ]:
import datetime
now = datetime.datetime.now()
file_date = now.strftime("%y%m%d")
#file_date='220919'
recueil_filename = f'data/corpus-work/{file_date}-RecueilBrut.csv'
recueil_complet_filename = f'data/corpus/{file_date}-RecueilComplet.csv'
print(recueil_filename)
print(recueil_complet_filename)

In [ ]:
cd ..

In [ ]:
import importlib
import pandas as pd

In [ ]:
import patat.imp.importer

## 1 - Chargement recueil
Le recueil a été exporté en csv de googlesheet.

Nom de fichier : **AAMMJJ-RecueilBrut.csv**

In [ ]:
df_brut = pd.read_csv(recueil_filename)

In [ ]:
df_brut

In [ ]:
# Suppression des urls vides
df_brut = df_brut[df_brut['url'].notna()]

In [ ]:
df_brut.keys()

In [ ]:
labels = [
    'infox', 'signé',
    'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
    'points de vue différents présentés', 'écriture de qualité',
    'faits', 'opinions', 'fausse nouvelle', 'insinuations',
    'scientifique sulfureux', 'des pages et des pages',
    'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
    'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
]

In [ ]:
def normalize_label(l):
    if l=='oui':
        return 1.0
#    elif l=='non':
#        return 0.0
    else:
        return 0.0

In [ ]:
for label in labels:
    df_brut.loc[:,label]=df_brut[label].apply(normalize_label)

In [ ]:
for label in labels:
    df_brut = df_brut[df_brut[label].notna()]

In [ ]:
for label in labels:
    print(label,df_brut[df_brut[label].isna()].shape)

### Extraction des sites

In [ ]:
import urllib
def get_site(url):
    if type(url)==str:
        return urllib.parse.urlsplit(url).netloc

In [ ]:
df_brut.loc[:,'site'] = df_brut['url'].apply(get_site)

In [ ]:
df_brut

## 2 - Acquisition et Parsing Articles

In [ ]:
importlib.reload(patat.imp.importer)
imp = patat.imp.importer.Importer()

In [ ]:
df_brut

In [ ]:
df_crawl = df_brut

In [ ]:
def crawl_item(url):
    html = ''
    print(url)
    try:
        html = imp.get_webpage(url, use_cache=True)
    except:
        print('Exception :' + str(url))
    return html

In [ ]:
df_crawl['html']=df_crawl['url'].apply(crawl_item)

In [ ]:
import patat.imp.parser_params
import patat.imp.parser

In [ ]:
importlib.reload(patat.imp.parser_params)
importlib.reload(patat.imp.parser)
parser = patat.imp.parser.Parser()

In [ ]:
def parse_article(url,html,site):
    record = {'url':url,'title':'','article':'','words':0}
    if html:
        if parser.is_known_site(url):
            record = parser.parse_article(url,html,site)
            record['parse_method']='parse'
        else:
            record = parser.parse_raw_article(url,html)
            record['parse_method']='raw'
    
    return record

In [ ]:
%%time
# Attention ! Le parsing peut prendre du temps
parsed = df_crawl.apply(lambda row: parse_article(row['url'], row['html'], row['site']), axis=1)

In [ ]:
df_parsed = pd.DataFrame(list(parsed))

## 3 - Vérification du dataframe parsé

In [ ]:
len(df_parsed)

## 4 - Création Recueil Complet
Le recueil complet est exporté en csv.

Nom de fichier : **AAMMJJ-RecueilComplet.csv**

In [ ]:
df_parsed[df_parsed['article'].isna()]

In [ ]:
#df_complet = pd.merge(df_parsed,df_brut,on='url')

In [ ]:
df_complet = pd.concat([df_parsed,df_brut[labels],df_brut['site']],axis=1)

In [ ]:
len(df_complet)

In [ ]:
df_complet['url'].to_csv('complet.csv')
df_parsed['url'].to_csv('parsed.csv')
df_brut['url'].to_csv('brut.csv')

In [ ]:
df_complet[df_complet['url'].isna()]

In [ ]:
#df_parsed=df_parsed.rename(columns={'url':'url_parsed'})
#df_complet=pd.concat([df_parsed,df_brut],axis=1)

In [ ]:
#df_complet['url_parsed']

In [ ]:
#df_complet[df_complet['url']!=df_complet['url_parsed']]

In [ ]:
# Articles vides
df_complet[df_complet['article']=='']

In [ ]:
# Suppression des articles vides
df_complet = df_complet[df_complet['article']!='']

In [ ]:
# Titres vides
df_complet[df_complet['title']=='']

In [ ]:
# Suppression des titres vides
df_complet = df_complet[df_complet['title']!='']

In [ ]:
#df_complet.drop('html',axis=1,inplace=True)

In [ ]:
# Concaténation Titre Abstract Article
df_complet.loc[:,'text'] = df_complet['title']+'\n'+df_complet['article']

In [ ]:
df_complet.to_csv(recueil_complet_filename,index=False)

# Bricolages

In [ ]:
df_complet.loc[440:]

In [ ]:
df_parsed.loc[:,'url']=df_parsed['url_parsed']

In [ ]:
pd.merge(df_parsed,df_brut)
#df_parsed.join(df_b,on='url')

In [ ]:
df_brut.keys()

In [ ]:
df_b = df_brut.loc[:,['url','explication', 'infox', 'signé',
       'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
       'points de vue différents présentés', 'écriture de qualité',
       'intention', 'faits', 'opinions', 'fausse nouvelle', 'insinuations',
       'exagération', 'scientifique sulfureux', 'des pages et des pages',
       'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
       'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
       'dénigrement', 'degré infox', 'observateur', 'thème (abandonné)',
       'type infox (abandonné)', 'Date de collection (abandonné)']]

In [ ]:
import patat.app.df_viewer

In [ ]:
viewer = patat.app.df_viewer.DfViewer()

In [ ]:
tree = viewer.get_tree_page()

In [ ]:
tree['tree'][1]

In [ ]:
df_complet

In [ ]:
df_parsed['url'] = df_parsed['url_parsed']

In [ ]:
df_brut['url']

In [ ]:
df_complet['article'][1]

In [ ]:
parser.clean_article(df_complet['article'][1])